In [50]:
import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd

In [51]:
def convert_csv_to_dict(csv_name):
    dict_from_csv = {}

    # with open(csv_name, mode='r') as csvfile:
    #     reader = csv.reader(csvfile)
    #     dict_from_csv = {rows[0]:rows[22] for rows in reader}
    # dict_from_csv = pd.read_csv(csv_name, header=None, index_col=0, squeeze=True).to_dict()

    # with open(csv_name, 'rb') as csvfile:
    #     reader = csv.reader(csvfile)
    #     dict_from_csv = {c[0]: c[1:] for c in zip(*reader)}

    columns = []
    with open(csv_name,'r') as csvfile: 
        reader = csv.reader(csvfile)
        for row in reader:
            if columns:
                for i, value in enumerate(row):
                    columns[i].append(value)
            else:
                # first row
                columns = [[value] for value in row]

    # you now have a column-major 2D array of your file.
    dict_from_csv = {c[0] : c[1:] for c in columns}

    return dict_from_csv

In [66]:
def get_movies(dict_from_csv):
    movies = {}
    movies['movie_id'] = []
    movies['movie_name'] = []
    for i in range(len(dict_from_csv['movie_id'])):
        if dict_from_csv['movie_id'][i] not in movies['movie_id']:
            movies['movie_id'].append(dict_from_csv['movie_id'][i])
            movies['movie_name'].append(dict_from_csv['original_title'][i])
    
    print(len(movies['movie_id']))
    return movies

In [67]:
def find_similarity(dict_from_csv, movie_id):
    find = False
    recommendation = []
    for i in range(len(dict_from_csv['track_id'])):
        if dict_from_csv['movie_id'][i] != movie_id and find == True:
            break
        if dict_from_csv['movie_id'][i] == movie_id:
            soundtrack_feature = [dict_from_csv['acousticness'][i], dict_from_csv['danceability'][i], dict_from_csv['energy'][i], dict_from_csv['instrumentalness'][i], dict_from_csv['liveness'][i], dict_from_csv['loudness'][i], dict_from_csv['speechiness'][i], dict_from_csv['tempo'][i], dict_from_csv['valence'][i]]
            soundtrack_feature = np.array(soundtrack_feature, dtype=float)
            ###
            dict_from_csv['similarity'] = []
            for i in range(len(dict_from_csv['track_id'])):
                sample_feature = [dict_from_csv['acousticness'][i], dict_from_csv['danceability'][i], dict_from_csv['energy'][i], dict_from_csv['instrumentalness'][i], dict_from_csv['liveness'][i], dict_from_csv['loudness'][i], dict_from_csv['speechiness'][i], dict_from_csv['tempo'][i], dict_from_csv['valence'][i]]
                sample_feature = np.array(sample_feature, dtype=float)
                similarity_score = dot(soundtrack_feature, sample_feature)/(norm(soundtrack_feature)*norm(sample_feature))
                # print(similarity_scores)
                dict_from_csv['similarity'].append(similarity_score)

            dataframe = pd.DataFrame.from_dict(dict_from_csv)
            dataframe = dataframe.sort_values('similarity', ascending=False).head(3)
            for index in dataframe.index:
                recommendation.append(index)

            # return dataframe
            ###
            find = True
        # soundtrack_feature.append(soundtrack)
        
    return recommendation

    # print(len(soundtrack_feature))
    # print(soundtrack_feature)
    # soundtrack_feature_list = np.array(soundtrack_feature, dtype=float)

    # dict_from_csv['similarity'] = []
    # for i in range(len(dict_from_csv['track_id'])):
    #     sample_feature = [dict_from_csv['acousticness'][i], dict_from_csv['danceability'][i], dict_from_csv['energy'][i], dict_from_csv['instrumentalness'][i], dict_from_csv['liveness'][i], dict_from_csv['loudness'][i], dict_from_csv['speechiness'][i], dict_from_csv['tempo'][i], dict_from_csv['valence'][i]]
    #     sample_feature_list = np.array(sample_feature, dtype=float)
    #     similarity_scores = soundtrack_feature_list.dot(sample_feature_list) / (np.linalg.norm(soundtrack_feature_list, axis=1) * np.linalg.norm(sample_feature_list))
    #     # print(similarity_scores)
    #     similarity_scores_list = np.array(similarity_scores, dtype=float)
    #     average_sim_score = np.average(similarity_scores_list)
    #     dict_from_csv['similarity'].append(average_sim_score)

    # dataframe = pd.DataFrame.from_dict(dict_from_csv)
    # return dataframe
# sim = dot(liked, sample)/(norm(liked)*norm(sample))
# dict_from_csv['similarity'].append(sim)


In [68]:
def recommendation_candidate(dict_from_csv, movies):
    candidates = {}
    candidates['movie_id'] = []
    candidates['movie_name'] = []
    candidates['recommendations'] = []
    for i in range(len(movies['movie_id'])):
        if i % 100 == 0:
            print(str(i) + " movies completed.")
        recommendation = find_similarity(dict_from_csv, movies['movie_id'][i])
        candidates['movie_id'].append(movies['movie_id'][i])
        candidates['movie_name'].append(movies['movie_name'][i])
        candidates['recommendations'].append(recommendation)

    candidates_df = pd.DataFrame.from_dict(candidates)
    candidates_df.to_csv("Candidates_recommendations.csv")

    return candidates

In [69]:
dict_from_csv = convert_csv_to_dict("movie-music.csv")
# recommendation = find_similarity(dict_from_csv, "Toy Story")
movies = get_movies(dict_from_csv)
# print(movies)
# print(recommendation)
candidates = recommendation_candidate(dict_from_csv, movies)

7701


KeyboardInterrupt: 